In [ ]:
# Install necessary packages
install.packages("keras")
install.packages("tensorflow")
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Load libraries
library(keras)
library(tensorflow)
library(caret)

In [ ]:
# Load your data
data <- read.csv("/content/SNVpcadata112.csv")

# Check data structure and summary
str(data)
summary(data)

'data.frame':	1320 obs. of  11 variables:
 $ Groups: num  12.3 12.3 12.3 12.3 12.3 ...
 $ PC1   : num  18.5 24.4 31.4 39.9 46.7 ...
 $ PC2   : num  1.35 2.65 1.16 1.11 5.65 ...
 $ PC3   : num  3.44 7.22 6.58 12.37 1.54 ...
 $ PC4   : num  -13.5 -10.6 -13.4 -21.7 -10.9 ...
 $ PC5   : num  8.04 8.37 7.77 14.5 3.52 ...
 $ PC6   : num  -1.806 0.274 -2.26 -2.244 -1.771 ...
 $ PC7   : num  1.141 3.471 1.756 -1.606 0.907 ...
 $ PC8   : num  -3.35 -6.01 -6.61 -6.49 -1.26 ...
 $ PC9   : num  1.027 1.262 -0.259 0.295 5.716 ...
 $ PC10  : num  -2.897 -1.512 0.104 -1.378 3.248 ...


     Groups           PC1                PC2                PC3          
 Min.   :10.58   Min.   :-60.3872   Min.   :-42.3482   Min.   :-28.8526  
 1st Qu.:12.32   1st Qu.:-17.6568   1st Qu.: -8.6752   1st Qu.: -5.8679  
 Median :21.30   Median : -0.6547   Median :  0.5938   Median : -0.4768  
 Mean   :20.28   Mean   :  0.0000   Mean   :  0.0000   Mean   :  0.0000  
 3rd Qu.:25.55   3rd Qu.: 16.1322   3rd Qu.:  8.7328   3rd Qu.:  5.5009  
 Max.   :29.40   Max.   : 74.5124   Max.   : 31.7772   Max.   : 26.8101  
      PC4                PC5                 PC6                PC7          
 Min.   :-23.8401   Min.   :-24.47765   Min.   :-27.4505   Min.   :-23.1754  
 1st Qu.: -4.7322   1st Qu.: -4.77663   1st Qu.: -2.8807   1st Qu.: -3.6055  
 Median :  0.9969   Median :  0.03676   Median :  0.8984   Median :  0.2349  
 Mean   :  0.0000   Mean   :  0.00000   Mean   :  0.0000   Mean   :  0.0000  
 3rd Qu.:  5.9040   3rd Qu.:  5.18415   3rd Qu.:  4.2291   3rd Qu.:  3.5689  
 Max.   : 22.1

In [ ]:
# Convert target variable to binary (0/1) format
data$Group <- as.factor(data$Group)
target <- as.numeric(data$Group) - 1  # Convert to binary 0/1

# Normalize the features
features <- scale(data[, -which(names(data) == "Group")])  # Exclude the 'Group' column which is the target

In [ ]:
# Split the data into training and testing sets
set.seed(123)
index <- sample(1:nrow(features), 0.8 * nrow(features))
train_features <- features[index, ]
train_target <- target[index]
test_features <- features[-index, ]
test_target <- target[-index]


In [ ]:
# Reshape the data for CNN input (add a dimension for the channel)
train_features <- array_reshape(train_features, c(nrow(train_features), ncol(train_features), 1))
test_features <- array_reshape(test_features, c(nrow(test_features), ncol(test_features), 1))


In [ ]:
# Define the model
model <- keras_model_sequential() %>%
  layer_conv_1d(filters = 64, kernel_size = 3, activation = 'relu',
                input_shape = c(ncol(train_features), 1),
                kernel_regularizer = regularizer_l2(l = 0.02), # Pass l = regularization factor
                padding = 'same') %>%
  layer_max_pooling_1d(pool_size = 2) %>%
  layer_dropout(rate = 0.3) %>%
  layer_conv_1d(filters = 128, kernel_size = 3, activation = 'relu',
                kernel_regularizer = regularizer_l2(l = 0.02), # Pass l = regularization factor
                padding = 'same') %>%
  layer_max_pooling_1d(pool_size = 2) %>%
  layer_dropout(rate = 0.3) %>%
  layer_flatten() %>%
  layer_dense(units = 64, activation = 'relu',
              kernel_regularizer = regularizer_l2(l = 0.02)) %>% # Pass l = regularization factor
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 1, activation = 'sigmoid')  # Change to binary classification output

ERROR: L2.__init__() got an unexpected keyword argument 'l'

In [ ]:
# Compile the model
model %>% compile(
  loss = 'binary_crossentropy',  # Use binary cross-entropy for binary classification
  optimizer = optimizer_adam(learning_rate = 0.001),
  metrics = 'accuracy'
)
# Print the model summary
model %>% summary()

ERROR: Error in eval(expr, envir, enclos): object 'model' not found


In [ ]:
# Define Early Stopping callback
early_stopping <- callback_early_stopping(
  monitor = "val_loss",
  patience = 100,
  restore_best_weights = TRUE
)


In [ ]:
# Train the model with validation data and Early Stopping
history <- model %>% fit(
  x = train_features,
  y = train_target,
  validation_split = 0.2,
  epochs = 1500,
  batch_size = 64,
  callbacks = list(early_stopping)
)

In [ ]:
# Evaluate the model on test data
evaluation <- model %>% evaluate(test_features, test_target)
cat("Test loss:", evaluation[[1]], "\n")
cat("Test accuracy:", evaluation[[2]], "\n")

Test loss: 0.3030628 
Test accuracy: 0.924812 


In [ ]:
# Make predictions on the test set
test_predictions <- model %>% predict(test_features)
test_predictions_classes <- ifelse(test_predictions > 0.5, 1, 0)  # Convert probabilities to binary classes


In [ ]:
# Ensure the predictions and target have the same length
stopifnot(length(test_predictions_classes) == length(test_target))

# Convert to factor for caret
test_target_factor <- as.factor(test_target)
test_predictions_classes_factor <- as.factor(test_predictions_classes)

In [ ]:
# Calculate confusion matrix using caret
conf_matrix <- confusionMatrix(test_predictions_classes_factor, test_target_factor)

# Print confusion matrix
print(conf_matrix)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 27  2
         1  8 96
                                          
               Accuracy : 0.9248          
                 95% CI : (0.8661, 0.9634)
    No Information Rate : 0.7368          
    P-Value [Acc > NIR] : 3.349e-08       
                                          
                  Kappa : 0.7948          
                                          
 Mcnemar's Test P-Value : 0.1138          
                                          
            Sensitivity : 0.7714          
            Specificity : 0.9796          
         Pos Pred Value : 0.9310          
         Neg Pred Value : 0.9231          
             Prevalence : 0.2632          
         Detection Rate : 0.2030          
   Detection Prevalence : 0.2180          
      Balanced Accuracy : 0.8755          
                                          
       'Positive' Class : 0               
                                    